# Optimizing Parameters in ExemPy

This notebook will show you how to fit parameters for attention weighting, building on the concepts we saw in ExemPy-Basics.ipynb

In [1]:
%load_ext autoreload
from ExemPy import *
from ExemPy.utils import *
from ExemPy.viz import *
from ExemPy.GCM import *
%aimport ExemPy, ExemPy.utils, ExemPy.viz, ExemPy.GCM
%autoreload 1
import math
import random
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.optimize import minimize
import seaborn as sns
sns.set(style='ticks', context='paper')
colors=["#e3c934","#68c4bf","#c51000","#287271"]
sns.set_palette(colors)

## Read in data, set initial parameters

In [2]:
pb52 = pd.read_csv('pb52_data//pb52.csv')
pb52 = HzToBark(pb52, ["F0", "F1", "F2", "F3"])
pb52.sample(5)

,type,gender,speaker,vowel,repetition,F0,F1,F2,F3,z0,z1,z2,z3
1240,c,f,63,FLEECE,1,290,320,3500,4260,2.925511,3.232807,16.655897,17.831833
1110,w,f,56,PALM,1,225,834,1282,2800,2.230755,7.472699,10.071610,15.240588
301,m,m,16,FLEECE,2,150,300,2460,3280,1.375924,3.028850,14.391403,16.251832
227,m,m,12,TRAP,2,133,630,1680,2280,1.173645,5.991351,11.843846,13.886698
1062,w,f,54,KIT,1,228,319,2500,3020,2.263729,3.222694,14.498027,15.728273


In [3]:
dimsvals={'z0' : 1,
          'z1' : .761,
          'z2' : .681,
          'z3' : .407}
dimslist = list(dimsvals.keys())

catslist = ['type', 'vowel']           # man, woman, or child; lexical set notation

cval = 25

exemplars = pb52.copy()

## Define error function
- In future versions, this function will be pre-defined in the library
- For now, there is some value in being able to tweak the function, and to see how it works!

In [4]:
def calcerror(x, test, exemplars, catslist, fitdims, cval, anchordim = None):
    '''
    Categorizes a data set and returns the proportion of stimuli/test rows
    that were categorized inaccurately. A lower value means a lower amount of
    error. Designed to be used with parameter fitting functions to assign
    values to attention weighting for the dimensions.
    
    Required paratemers:
    
    x = Array. Initial guesses for parameters
    
    test = DataFrame. Stimuli to be categorized
    
    exemplars = DataFrame. Exemplar cloud to use for categorization
    
    catslist = List of strings. Each string should correspond to a category that
        should be assigned to the test
    
    fitdims = List of strings. Each string should correspond to a dimension
        for which parameters should be fit.
        
    Optional parameters:
    
    anchordim = String. Dimension for parameter which will not be fit, but will
        instead be hard-coded as 1. This helps constrain the set of possible
        solutions
    
    
    '''
    #x = [z1,z2,z3]
    # dimslist = [z0, z1, z2, z3]
    # dimsdict = {z0 =1, z1 = z1guess, z2 = z2guess, z3=z3guess}
    # dimslist[0] = z0        dl[1] = z1,       dl[2] = z2.      dl[3] = z3
    #            1            x[0]= z1,          x[1] = z2,      x[2] =z3 

    
    dimsvals = {fitdims[i]: x[i] for i in range(len(fitdims))}
    if anchordim != None:
        dimsvals.update({anchordim:1})
    
    choices = multicat(test, cloud, catslist, dimsvals, cval)
    accuracy = checkaccuracy(choices, catslist)
    category = catslist[0]
    err = accuracy[category+"Acc"].value_counts(normalize=True)['n']
    return err

## Specify arguments for optimization

In [5]:
fitdims = dimslist[1:]      # Fit all dimensions except item 0
anchordim = dimslist[0]     # Set item 0 to 1

name = 'pb52-062823'                # name of output spreadsheet
nt = 3                        # number of times that random x is generated
t = 0.1                     # Tolerance value -- lower = more evals

# fit based on 50 exemplars of each vowel
test = gettestset(exemplars, "vowel", 1)

cloud = exemplars
cats = ["vowel"]


## Optimize!
You may want to come back later for the results: 
- Results will be saved to a spreadsheet. (\[NAME\].csv)
- Settings will be saved to a text file. (\[NAME\]\_info.txt)

In [ ]:
# Initialize lists
resultslist=[['start','fit','error','evals']]
wlist=[]

print("----- Parameters -----")
if anchordim != None: 
    print("Anchored (1):  ", anchordim)
    
print("Optimized:     ", fitdims)
print("")
print("Categorizing for: ", cats)
print("")
print("Trials: ", nt)
print("")

for i in range(0,nt): 
    x=np.divide(random.sample(range(0,300),len(fitdims)),100)
        # Get a random sample of numbers between 0 and 300,
           # divide by 100 to get floats between 0 and 3  
    xguess = x
    result = minimize(calcerror,
                  xguess,  # the initial guess array
                  args=(test, cloud, cats, fitdims, cval, anchordim), # arguments for the error function
                  method='Powell',  
                  tol = t)  # a 'tolerance' value, smaller means more function evaluation, but potentially better fit
    
    # Create list to save as csv
    start = x
    fit = np.round(result.x,3)
    error = result.fun
    evals = result.nfev
    row = [start,fit,error,evals]
    resultslist.append(row)
    
    # Re-compose w dict to save with json  
    wdict_keys = fitdims
    wdict_vals = list(fit)
    #if anchordim != None: 
    wdict_keys.insert(0, anchordim)
    wdict_vals.insert(0, 'hi')    
    wdict = {wdict_keys[i]: wdict_vals[i] for i in range(len(wdict_keys))}
    wlist.append(wdict)
    
    print ("-----", (i+1) ," -----")
    print("Initial guess:    ", start)
    print("Optimized:        ", fit)
    print(" ")
    print("Number evals: ", evals)
    print("Error:        ", error)
    print("")
results=pd.DataFrame(resultslist)
results.columns = results.iloc[0]
results=results[1:]

settings = {"fitdims": fitdims, "anchordim": anchordim, "cats": cats, "trials":nt, "tol": t }

# write results to csv
### good for if you want to leave it running while you do something else!
results.to_csv(name+".csv")     
with open((name+"_info.txt"),"w") as file:
    file.write(str(settings))
    
# Save w list
with open(name, "w") as fout:
    json.dump(wlist, fout)

#Clear lists
resultslist = []

In [ ]:
wlist